In [3]:

import sys
import os

# Add the parent directory (project root) to the system path
sys.path.append(os.path.dirname(os.getcwd()))

# Now we can import from src
from src.data_loader import load_news_data, validate_news_data
from src.config import TICKERS

✅ Configuration loaded
📊 Analyzing 6 companies: ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
📈 Technical analysis directory: d:\10 acadamy\Financial_News_Stock_Analysis\data\technical
😊 Sentiment analysis directory: d:\10 acadamy\Financial_News_Stock_Analysis\data\sentiment


In [8]:
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuration from your setup
TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
TECHNICAL_DIR = 'd:/10 acadamy/Financial_News_Stock_Analysis/data/technical'
SENTIMENT_DIR = 'd:/10 acadamy/Financial_News_Stock_Analysis/data/sentiment'
PROCESSED_DIR = '../data/processed'

print("🚀 Starting Feature Engineering Pipeline...")
print(f"Tickers: {TICKERS}")
print(f"Technical data: {TECHNICAL_DIR}")
print(f"Sentiment data: {SENTIMENT_DIR}")

🚀 Starting Feature Engineering Pipeline...
Tickers: ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
Technical data: d:/10 acadamy/Financial_News_Stock_Analysis/data/technical
Sentiment data: d:/10 acadamy/Financial_News_Stock_Analysis/data/sentiment


In [24]:
def create_sample_data():
    """Create sample technical and sentiment data for testing"""
    print("📝 Creating sample data for feature engineering...")
    
    # Create date range
    dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
    
    sample_data = []
    
    for ticker in TICKERS:
        # Base price for each stock
        base_price = {
            'AAPL': 150, 'AMZN': 130, 'GOOG': 100, 
            'META': 250, 'MSFT': 300, 'NVDA': 400
        }.get(ticker, 100)
        
        for i, date in enumerate(dates):
            # Generate realistic price data with some trend and noise
            trend = i * 0.1
            noise = np.random.normal(0, 2)
            price = base_price + trend + noise
            
            # Ensure prices are positive
            price = max(10, price)
            
            # Generate OHLC data
            open_price = price * (1 + np.random.normal(0, 0.01))
            high = max(open_price, price) * (1 + abs(np.random.normal(0, 0.02)))
            low = min(open_price, price) * (1 - abs(np.random.normal(0, 0.02)))
            close = price
            volume = np.random.randint(1000000, 10000000)
            
            # Generate sentiment data with some correlation to returns
            base_sentiment = np.random.normal(0, 0.1)
            # Add some correlation with price movements
            if i > 0:
                prev_close = sample_data[-1]['close'] if sample_data else base_price
                price_change = (close - prev_close) / prev_close
                sentiment = base_sentiment + price_change * 0.3
            else:
                sentiment = base_sentiment
            
            sample_data.append({
                'date': date,
                'ticker': ticker,
                'open': open_price,
                'high': high,
                'low': low,
                'close': close,
                'volume': volume,
                'daily_compound_mean': sentiment,
                'positive': max(0, sentiment),
                'negative': min(0, -sentiment),
                'neutral': 1 - abs(sentiment)
            })
    
    df = pd.DataFrame(sample_data)
    print(f"✅ Created sample data: {len(df)} rows, {df['ticker'].nunique()} tickers")
    return df

# Create sample data
merged_data = create_sample_data()

print(f"\n📊 Sample data created successfully!")
print(f"Shape: {merged_data.shape}")
print(f"Columns: {merged_data.columns.tolist()}")
print(f"Date range: {merged_data['date'].min()} to {merged_data['date'].max()}")
print(f"Tickers: {merged_data['ticker'].unique()}")

📝 Creating sample data for feature engineering...
✅ Created sample data: 2196 rows, 6 tickers

📊 Sample data created successfully!
Shape: (2196, 11)
Columns: ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'daily_compound_mean', 'positive', 'negative', 'neutral']
Date range: 2023-01-01 00:00:00 to 2024-01-01 00:00:00
Tickers: ['AAPL' 'AMZN' 'GOOG' 'META' 'MSFT' 'NVDA']


In [16]:
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
TICKERS = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
TECHNICAL_DIR = 'd:/10 acadamy/Financial_News_Stock_Analysis/data/technical'
SENTIMENT_DIR = 'd:/10 acadamy/Financial_News_Stock_Analysis/data/sentiment'
PROCESSED_DIR = '../data/processed'

print("🚀 Starting Feature Engineering Pipeline...")
print(f"Tickers: {TICKERS}")

# Create directories
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Remove the undefined merged_dir line and replace with proper data loading
def load_or_create_data():
    """Load existing data or create sample data for feature engineering"""
    print("📊 Checking for existing data...")
    
    # Check if we have saved merged data from previous steps
    merged_data_path = os.path.join(PROCESSED_DIR, "sentiment_returns_merged.csv")
    
    if os.path.exists(merged_data_path):
        print(f"✅ Loading existing merged data from: {merged_data_path}")
        merged_data = pd.read_csv(merged_data_path)
        print(f"Loaded: {len(merged_data)} rows, {merged_data['ticker'].nunique()} tickers")
        return merged_data
    else:
        print("❌ No existing merged data found. Creating sample data...")
        return create_sample_data()

def create_sample_data():
    """Create sample technical and sentiment data for testing"""
    print("📝 Creating sample data for feature engineering...")
    
    # Create date range
    dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
    
    sample_data = []
    
    for ticker in TICKERS:
        # Base price for each stock
        base_price = {
            'AAPL': 150, 'AMZN': 130, 'GOOG': 100, 
            'META': 250, 'MSFT': 300, 'NVDA': 400
        }.get(ticker, 100)
        
        for i, date in enumerate(dates):
            # Generate realistic price data with some trend and noise
            trend = i * 0.1
            noise = np.random.normal(0, 2)
            price = base_price + trend + noise
            
            # Ensure prices are positive
            price = max(10, price)
            
            # Generate OHLC data
            open_price = price * (1 + np.random.normal(0, 0.01))
            high = max(open_price, price) * (1 + abs(np.random.normal(0, 0.02)))
            low = min(open_price, price) * (1 - abs(np.random.normal(0, 0.02)))
            close = price
            volume = np.random.randint(1000000, 10000000)
            
            # Generate sentiment data with some correlation to returns
            base_sentiment = np.random.normal(0, 0.1)
            # Add some correlation with price movements
            if i > 0:
                prev_close = sample_data[-1]['close'] if sample_data else base_price
                price_change = (close - prev_close) / prev_close
                sentiment = base_sentiment + price_change * 0.3
            else:
                sentiment = base_sentiment
            
            sample_data.append({
                'date': date,
                'ticker': ticker,
                'open': open_price,
                'high': high,
                'low': low,
                'close': close,
                'volume': volume,
                'daily_compound_mean': sentiment,
                'positive': max(0, sentiment),
                'negative': min(0, -sentiment),
                'neutral': 1 - abs(sentiment)
            })
    
    df = pd.DataFrame(sample_data)
    print(f"✅ Created sample data: {len(df)} rows, {df['ticker'].nunique()} tickers")
    return df

# Load or create data
merged_data = load_or_create_data()

print(f"\n📊 Data loaded successfully!")
print(f"Shape: {merged_data.shape}")
print(f"Columns: {merged_data.columns.tolist()}")
print(f"Date range: {merged_data['date'].min()} to {merged_data['date'].max()}")
print(f"Tickers: {merged_data['ticker'].unique()}")

🚀 Starting Feature Engineering Pipeline...
Tickers: ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']
📊 Checking for existing data...
✅ Loading existing merged data from: ../data/processed\sentiment_returns_merged.csv
Loaded: 4000 rows, 8 tickers

📊 Data loaded successfully!
Shape: (4000, 6)
Columns: ['date', 'daily_compound_mean', 'ticker', 'close', 'return', 'next_return']
Date range: 2023-11-27 to 2025-11-21
Tickers: ['AAPL' 'GOOGL' 'MSFT' 'TSLA' 'AMZN' 'META' 'NFLX' 'NVDA']


In [21]:
def compute_technical_indicators(df):
    """Compute comprehensive technical indicators - FIXED VERSION"""
    print("🔧 Computing technical indicators...")
    
    df = df.copy()
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    
    results = []
    
    for ticker in df['ticker'].unique():
        print(f"  Processing {ticker}...")
        ticker_data = df[df['ticker'] == ticker].copy()
        
        # Price-based features
        if 'close' in ticker_data.columns:
            # Returns
            ticker_data['return'] = ticker_data['close'].pct_change()
            ticker_data['log_return'] = np.log(ticker_data['close'] / ticker_data['close'].shift(1))
            
            # Moving averages - FIXED: Use direct computation without reset_index
            for window in [5, 10, 20, 50]:
                ticker_data[f'sma_{window}'] = ticker_data['close'].rolling(window=window).mean()
            
            # Exponential moving averages
            ticker_data['ema_12'] = ticker_data['close'].ewm(span=12).mean()
            ticker_data['ema_26'] = ticker_data['close'].ewm(span=26).mean()
            
            # Price position relative to moving averages
            ticker_data['price_vs_sma_20'] = (ticker_data['close'] - ticker_data['sma_20']) / ticker_data['sma_20']
            ticker_data['price_vs_sma_50'] = (ticker_data['close'] - ticker_data['sma_50']) / ticker_data['sma_50']
            
            # Volatility
            ticker_data['volatility_5d'] = ticker_data['return'].rolling(5).std()
            ticker_data['volatility_20d'] = ticker_data['return'].rolling(20).std()
            
            # High-Low features
            if all(col in ticker_data.columns for col in ['high', 'low']):
                ticker_data['high_low_range'] = (ticker_data['high'] - ticker_data['low']) / ticker_data['close']
                ticker_data['high_low_ratio'] = ticker_data['high'] / ticker_data['low']
        
        # Volume features
        if 'volume' in ticker_data.columns:
            ticker_data['volume_sma_5'] = ticker_data['volume'].rolling(5).mean()
            ticker_data['volume_sma_20'] = ticker_data['volume'].rolling(20).mean()
            ticker_data['volume_ratio_5'] = ticker_data['volume'] / ticker_data['volume_sma_5']
            ticker_data['volume_ratio_20'] = ticker_data['volume'] / ticker_data['volume_sma_20']
            
            # Volume-price relationship
            if 'return' in ticker_data.columns:
                ticker_data['volume_price_trend'] = ticker_data['volume'] * ticker_data['return'].abs()
        
        # RSI calculation
        if 'close' in ticker_data.columns:
            def compute_rsi(series, period=14):
                delta = series.diff()
                gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
                loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
                rs = gain / loss
                rsi = 100 - (100 / (1 + rs))
                return rsi
            
            ticker_data['rsi_14'] = compute_rsi(ticker_data['close'], 14)
        
        # MACD calculation
        if 'close' in ticker_data.columns:
            def compute_macd(series):
                ema_fast = series.ewm(span=12).mean()
                ema_slow = series.ewm(span=26).mean()
                macd = ema_fast - ema_slow
                macd_signal = macd.ewm(span=9).mean()
                macd_histogram = macd - macd_signal
                return macd, macd_signal, macd_histogram
            
            macd, signal, hist = compute_macd(ticker_data['close'])
            ticker_data['macd'] = macd
            ticker_data['macd_signal'] = signal
            ticker_data['macd_hist'] = hist
        
        # Bollinger Bands
        if 'close' in ticker_data.columns:
            def compute_bollinger_bands(series, period=20, std=2):
                sma = series.rolling(window=period).mean()
                rolling_std = series.rolling(window=period).std()
                upper_band = sma + (rolling_std * std)
                lower_band = sma - (rolling_std * std)
                bandwidth = (upper_band - lower_band) / sma
                position = (series - lower_band) / (upper_band - lower_band)
                return upper_band, lower_band, bandwidth, position
            
            upper, lower, width, pos = compute_bollinger_bands(ticker_data['close'])
            ticker_data['bb_upper'] = upper
            ticker_data['bb_lower'] = lower
            ticker_data['bb_width'] = width
            ticker_data['bb_position'] = pos
        
        results.append(ticker_data)
    
    # Combine all tickers
    df_final = pd.concat(results, ignore_index=True)
    print("✅ Technical indicators computed successfully!")
    return df_final

# Apply technical indicators
print("\n🚀 Applying technical indicators...")
merged_data = compute_technical_indicators(merged_data)

# Display results
print(f"\n🎯 TECHNICAL INDICATORS APPLIED SUCCESSFULLY!")
print(f"Final data shape: {merged_data.shape}")
print(f"Total columns: {len(merged_data.columns)}")

# Show the technical columns that were added
tech_columns = [col for col in merged_data.columns if any(x in col for x in 
                ['sma', 'ema', 'rsi', 'macd', 'bb', 'volatility', 'return'])]
print(f"Technical columns created: {len(tech_columns)}")
print(f"Sample technical columns: {tech_columns[:10]}...")

# Display basic statistics
print(f"\n📊 BASIC STATISTICS:")
print(f"Mean return: {merged_data['return'].mean():.6f}")
print(f"Return std: {merged_data['return'].std():.6f}")
if 'rsi_14' in merged_data.columns:
    print(f"RSI range: {merged_data['rsi_14'].min():.2f} - {merged_data['rsi_14'].max():.2f}")
print(f"Data points per ticker: {merged_data.groupby('ticker').size().to_dict()}")


🚀 Applying technical indicators...
🔧 Computing technical indicators...
  Processing AAPL...
  Processing AMZN...
  Processing GOOGL...


  Processing META...
  Processing MSFT...
  Processing NFLX...
  Processing NVDA...
  Processing TSLA...
✅ Technical indicators computed successfully!

🎯 TECHNICAL INDICATORS APPLIED SUCCESSFULLY!
Final data shape: (4000, 25)
Total columns: 25
Technical columns created: 21
Sample technical columns: ['return', 'next_return', 'log_return', 'sma_5', 'sma_10', 'sma_20', 'sma_50', 'ema_12', 'ema_26', 'price_vs_sma_20']...

📊 BASIC STATISTICS:
Mean return: 0.001546
Return std: 0.024586
RSI range: 6.92 - 100.00
Data points per ticker: {'AAPL': 500, 'AMZN': 500, 'GOOGL': 500, 'META': 500, 'MSFT': 500, 'NFLX': 500, 'NVDA': 500, 'TSLA': 500}


In [22]:
def engineer_sentiment_features(df):
    """Create sentiment-based features"""
    print("\n😊 Engineering sentiment features...")
    
    df = df.copy()
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    
    results = []
    
    for ticker in df['ticker'].unique():
        ticker_data = df[df['ticker'] == ticker].copy()
        
        # Use daily_compound_mean as main sentiment column
        if 'daily_compound_mean' in ticker_data.columns:
            main_sentiment_col = 'daily_compound_mean'
            
            # Lagged sentiment features
            for lag in [1, 2, 3, 5]:
                ticker_data[f'sentiment_lag_{lag}'] = ticker_data[main_sentiment_col].shift(lag)
            
            # Rolling sentiment statistics
            for window in [3, 5, 10]:
                ticker_data[f'sentiment_rolling_mean_{window}'] = ticker_data[main_sentiment_col].rolling(window).mean()
                ticker_data[f'sentiment_rolling_std_{window}'] = ticker_data[main_sentiment_col].rolling(window).std()
            
            # Sentiment momentum
            ticker_data['sentiment_momentum_5'] = ticker_data[main_sentiment_col] - ticker_data[main_sentiment_col].shift(5)
            
            # Sentiment volatility
            ticker_data['sentiment_volatility_5d'] = ticker_data[main_sentiment_col].rolling(5).std()
            
            # Sentiment extremes
            ticker_data['sentiment_extreme_positive'] = (ticker_data[main_sentiment_col] > ticker_data[main_sentiment_col].rolling(20).quantile(0.8)).astype(int)
            ticker_data['sentiment_extreme_negative'] = (ticker_data[main_sentiment_col] < ticker_data[main_sentiment_col].rolling(20).quantile(0.2)).astype(int)
            
            # Sentiment-price interaction
            if 'return' in ticker_data.columns:
                ticker_data['sentiment_return_interaction'] = ticker_data[main_sentiment_col] * ticker_data['return']
            
            if 'volume' in ticker_data.columns:
                ticker_data['sentiment_volume_interaction'] = ticker_data[main_sentiment_col] * ticker_data['volume']
        
        results.append(ticker_data)
    
    df_final = pd.concat(results, ignore_index=True)
    print("✅ Sentiment features engineered")
    return df_final

def create_target_variables(df):
    """Create target variables for machine learning"""
    print("🎯 Creating target variables...")
    
    df = df.copy()
    df = df.sort_values(['ticker', 'date']).reset_index(drop=True)
    
    results = []
    
    for ticker in df['ticker'].unique():
        ticker_data = df[df['ticker'] == ticker].copy()
        
        if 'return' in ticker_data.columns:
            # Future returns for different horizons
            horizons = [1, 3, 5]  # 1-day, 3-day, 5-day ahead
            
            for horizon in horizons:
                # Future return
                ticker_data[f'target_return_{horizon}d'] = ticker_data['return'].shift(-horizon)
                
                # Binary classification: will price go up (1) or down (0)
                ticker_data[f'target_direction_{horizon}d'] = (ticker_data[f'target_return_{horizon}d'] > 0).astype(int)
        
        results.append(ticker_data)
    
    df_final = pd.concat(results, ignore_index=True)
    print("✅ Target variables created")
    return df_final

def finalize_features(df):
    """Final processing and cleaning of features"""
    print("🧹 Finalizing features...")
    
    df = df.copy()
    
    # Remove rows with too many NaN values (from rolling calculations)
    initial_rows = len(df)
    df = df.dropna()
    final_rows = len(df)
    
    print(f"   Rows before cleaning: {initial_rows:,}")
    print(f"   Rows after cleaning: {final_rows:,}")
    print(f"   Total columns: {len(df.columns)}")
    
    # Feature categories
    feature_categories = {
        'Price Features': [col for col in df.columns if any(x in col for x in ['close', 'high', 'low', 'sma', 'ema', 'bb', 'return', 'volatility'])],
        'Volume Features': [col for col in df.columns if 'volume' in col],
        'Technical Indicators': [col for col in df.columns if any(x in col for x in ['rsi', 'macd', 'bb'])],
        'Sentiment Features': [col for col in df.columns if any(x in col for x in ['sentiment', 'compound'])],
        'Target Variables': [col for col in df.columns if 'target' in col]
    }
    
    print("\n📊 FEATURE SUMMARY:")
    for category, features in feature_categories.items():
        print(f"   {category}: {len(features)} features")
    
    return df

# Apply the complete feature engineering pipeline
print("\n🔧 Running complete feature engineering pipeline...")

# Apply sentiment features
merged_data = engineer_sentiment_features(merged_data)

# Create target variables
merged_data = create_target_variables(merged_data)

# Final processing
features_df = finalize_features(merged_data)

print(f"\n🎉 FEATURE ENGINEERING COMPLETED!")
print(f"Final dataset shape: {features_df.shape}")
print(f"Total features: {len(features_df.columns)}")

# Save the engineered features
output_path = os.path.join(PROCESSED_DIR, "engineered_features.csv")
features_df.to_csv(output_path, index=False)
print(f"💾 Engineered features saved to: {output_path}")

# Display sample of the final data
print(f"\n📋 Sample of engineered features:")
sample_cols = ['ticker', 'date', 'close', 'return', 'sma_20', 'rsi_14', 'daily_compound_mean', 'sentiment_lag_1', 'target_return_1d']
available_cols = [col for col in sample_cols if col in features_df.columns]
print(features_df[available_cols].head(10))


🔧 Running complete feature engineering pipeline...

😊 Engineering sentiment features...
✅ Sentiment features engineered
🎯 Creating target variables...
✅ Target variables created
🧹 Finalizing features...
   Rows before cleaning: 4,000
   Rows after cleaning: 3,568
   Total columns: 46

📊 FEATURE SUMMARY:
   Price Features: 23 features
   Volume Features: 0 features
   Technical Indicators: 8 features
   Sentiment Features: 16 features
   Target Variables: 6 features

🎉 FEATURE ENGINEERING COMPLETED!
Final dataset shape: (3568, 46)
Total features: 46
💾 Engineered features saved to: ../data/processed\engineered_features.csv

📋 Sample of engineered features:
   ticker        date       close    return      sma_20     rsi_14  \
49   AAPL  2024-02-07  187.637970  0.000581  187.114400  51.600991   
50   AAPL  2024-02-08  186.558151 -0.005755  187.219904  42.806363   
51   AAPL  2024-02-09  187.321930  0.004094  187.393317  38.552551   
52   AAPL  2024-02-12  185.635666 -0.009002  187.466071 